# Brief Introduction

This is the 2023 revison of the **Social Media impact on the Crowdfunding Success** project. In this revison, there are a few changes made to the project. The changes are listed [here](https://www.evernote.com/shard/s458/sh/5feb1b0b-d2b6-d0d2-ce9e-1d65a87ebb81/).

In [1]:
import os
import nltk
import json
import spacy
import random
import gensim
import pyLDAvis
import pyLDAvis.gensim_models 
import warnings
import torch
import joblib
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import textdescriptives as td
from tqdm import tqdm
from nrclex import NRCLex
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
warnings.filterwarnings("ignore", category=DeprecationWarning)

torch.set_num_threads(4)

def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    pd.set_option('mode.chained_assignment', None)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)


current_dir = r'/workspaces/Crowdfunding-Social-Media-Drivers/Data/Original_dataset/'

nlp = spacy.load('en_core_web_trf')
nlp_td = spacy.load('en_core_web_sm')
nlp_td.add_pipe("textdescriptives/all")
stop = stopwords.words('english')
post_data = pd.read_csv(current_dir+"2022-07-22-09-31-10-IST-feed-download.csv")


#basic data cleaning
post_data.drop_duplicates(subset=['Message'], keep='last', inplace=True)
post_data.reset_index(inplace=True)
post_data = post_data.rename(columns={"index": "original_index"})
post_data.columns = [x.lower().replace(' ','_') for x in post_data.columns.to_list()]
post_data.drop(['user_name', 'facebook_id', 'page_category', 'page_admin_top_country',
                'overperforming_score_(weighted__—__likes_1x_shares_1x_comments_1x_love_1x_wow_1x_haha_1x_sad_1x_angry_1x_care_1x_)'],
                 axis=1, inplace=True)
post_data.head()


/home/codespace/.python/current/lib/python3.10/site-packages/textdescriptives/components/quality_data_classes.py:24: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  extra = Extra.forbid
/home/codespace/.python/current/lib/python3.10/site-packages/pydantic/_internal/_config.py:271: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/home/codespace/.python/current/lib/python3.10/site-packages/textdescriptives/components/quality_data_classes.py:64: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`).

,original_index,page_name,page_created,likes_at_posting,followers_at_posting,post_created,post_created_date,post_created_time,type,total_interactions,...,url,message,link,final_link,image_text,link_text,description,sponsor_id,sponsor_name,sponsor_category
0,0,GoFundMe,10-02-2010 02:10,1839689,1942958.0,2022-07-22 01:22:32 IST,22-07-2022,01:22:32,Photo,21,...,https://www.facebook.com/299947508549/posts/10...,“I knew I had to do this with as many veterans...,https://www.facebook.com/gofundme/photos/a.101...,NaN,HONOR,NaN,NaN,NaN,NaN,NaN
1,1,Kickstarter,21-04-2009 16:42,1589842,1587991.0,2022-07-22 01:18:36 IST,22-07-2022,01:18:36,Link,3,...,https://www.facebook.com/73182029884/posts/101...,Artist and designer Janos Stone is creating a ...,https://www.kickstarter.com/projects/haus/toyh...,https://www.kickstarter.com/projects/haus/toyh...,NaN,ToyHaus: a uniquely beautiful mini-playhouse f...,"ToyHaus is a washable, popup mini-playhouse wi...",NaN,NaN,NaN
2,2,Kickstarter,21-04-2009 16:42,1589842,1587991.0,2022-07-21 18:52:39 IST,21-07-2022,18:52:39,Link,9,...,https://www.facebook.com/73182029884/posts/101...,The inaugural meeting of Kickstarter’s Communi...,https://www.kickstarter.com/blog/reporting-bac...,https://www.kickstarter.com/blog/reporting-bac...,NaN,Reporting Back from the First Meeting of the K...,"Earlier this month, I was honored to host the ...",NaN,NaN,NaN
3,3,GoFundMe,10-02-2010 02:10,1839047,1942443.0,2022-07-21 00:23:03 IST,21-07-2022,00:23:03,Native Video,163,...,https://www.facebook.com/299947508549/posts/10...,This 25-year-old man rushed into a burning hou...,https://www.facebook.com/peoplemag/videos/2043...,NaN,NaN,People,"""Nicholas Bostic's heroic actions saved lives,...",NaN,NaN,NaN
4,4,Kickstarter,21-04-2009 16:42,1589866,1588006.0,2022-07-20 02:38:36 IST,20-07-2022,02:38:36,Link,7,...,https://www.facebook.com/73182029884/posts/101...,An occult-themed JRPG with a fully integrated ...,https://www.kickstarter.com/projects/cherrymoc...,https://www.kickstarter.com/projects/cherrymoc...,NaN,EXIT VEIL: Occult & Tarot JRPG,A Dark-Psychedelic JRPG & fully integrated Tar...,NaN,NaN,NaN


## Basic pre_processing and Feature engineering

* Text Complexity evaluation
* Sentiment Recognition (fear, anger, anticip, trust, surprise, positive, negative, sadness, disgust, joy)
* Entity Recognition
* Stopword Removal
* Lemmatization
* Sponsor effect investigation
* Outlier Removal

In [2]:
def get_complexity_scores(text_description):
    readability_scores = []
    lix_scores = []
    entropy_scores = []
    perplexity_scores = []

    print("Computing text descriptives...")
    for i in tqdm(range(len(text_description))):
        doc = nlp_td(text_description[i])
        readability = doc._.readability['automated_readability_index']
        readability_lix = doc._.readability['lix']
        entropy = doc._.entropy
        perplexity = doc._.perplexity

        #add to list
        readability_scores.append(readability)
        lix_scores.append(readability_lix)
        entropy_scores.append(entropy)
        perplexity_scores.append(perplexity)

    print('\n')

    return readability_scores, lix_scores, entropy_scores, perplexity_scores


def get_emotion_scores(text_description):
    emotions = ['fear', 'anger', 'anticip', 'trust', 'surprise', 'positive', 'negative', 'sadness', 'disgust', 'joy']
    emotion_scores = []

    for i in tqdm(range(len(text_description))):
        doc = NRCLex(text_description[i])
        emotion_scores.append([doc.affect_frequencies[x] for x in emotions])

    emotion_scores = np.array(emotion_scores)
    emotion_scores = pd.DataFrame(emotion_scores, columns=emotions)

    emotion_scores.columns

    for col in emotion_scores.columns:
        zero_idx = emotion_scores[emotion_scores[col] == 0].index
        emotion_scores.loc[zero_idx, col] = np.random.uniform(0.00001, 0.0001, len(zero_idx))
    
    return emotion_scores

# Extracting NERs out of the messages and other text bodies
def recognise_entities(col, entities):
    null_count = 0
    for i in tqdm(range(len(post_data))):
        try:
            doc = nlp(post_data[col][i])
            entities.append((i, [(ent.text, ent.label_) for ent in doc.ents]))
        except (TypeError, ValueError):
            null_count += 1

    print(f"Number of null values in {col}: {null_count}")
    return entities


def get_entities(current_dir, text_columns):
    if os.path.isfile(current_dir + "idx_to_list.json"):
        with open(current_dir + "idx_to_list.json", "r") as f:
            idx_to_list = json.load(f)

    else:
        if os.path.isfile(current_dir + "entities.json"):
            with open(current_dir + "entities.json", "r") as f:
                entities = json.load(f)
        else:
            entities = dict()
            for text_col in text_columns:
                temp_entities = []
                entities[text_col] = recognise_entities(text_col, temp_entities)

        indices = []

        for key in entities.keys():
            indices += [x[0] for x in entities[key]]

        indices = list(set(indices))

        idx_to_list = dict()
        for key in entities.keys():
            for idx, recognised_entity_list in entities[key]:
                if idx in idx_to_list.keys():
                    idx_to_list[idx] += recognised_entity_list
                else:
                    idx_to_list[idx] = recognised_entity_list
                idx_to_list[idx] = list(set(idx_to_list[idx]))

    return idx_to_list

def lemmatize_text(col):
    nlp = spacy.load("en_core_web_trf")
    processed_col = post_data[col].apply(nlp)
    processed_col = processed_col.apply(
        lambda x: [token.lemma_ for token in x if token.lemma_ not in stop]
    )
    processed_col = processed_col.apply(lambda x: " ".join(list(set(x))))
    return processed_col

In [3]:
text_docs = post_data.description.copy()
null_desc_idx = post_data[post_data.description.isnull()].index
text_docs[null_desc_idx] = post_data.loc[null_desc_idx, "message"]

readability_scores, lix_scores, entropy_scores, perplexity_scores = get_complexity_scores(text_docs)

post_data['readability'] = readability_scores
post_data['readability_lix'] = lix_scores
post_data['entropy_scores'] = entropy_scores
post_data['perplexity_scores'] = perplexity_scores

emotion_scores = get_emotion_scores(text_docs)
post_data = pd.concat([post_data, emotion_scores], axis=1)

Computing text descriptives...


  0%|          | 0/8983 [00:00<?, ?it/s]/home/codespace/.python/current/lib/python3.10/site-packages/textdescriptives/components/coherence.py:44: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarities.append(sent.similarity(sents[i + order]))
  0%|          | 11/8983 [00:00<02:39, 56.25it/s]

100%|██████████| 8983/8983 [02:00<00:00, 74.68it/s] 


100%|██████████| 8983/8983 [00:04<00:00, 1900.37it/s]


In [4]:
text_columns = set(["message", "image_text", "link_text", "description"])
for col in text_columns:
    post_data[col] = post_data[col].str.lower()
    post_data[col] = post_data[col].str.replace(r"http?.*.com?", " ", regex=True)
    post_data[col] = post_data[col].str.replace("[^\w\s]", "", regex=True)


all_entities = get_entities(current_dir, text_columns)
indices = sorted([int(key) for key in all_entities.keys()])
all_entity_types = [
    [y for x, y in entity] for entity in [all_entities[str(idx)] for idx in indices]
]

set_all_entities = []
[set_all_entities.extend(x) for x in all_entity_types]

unique_entity_types = list(set(set_all_entities))
entity_type_count = [(x, set_all_entities.count(x)) for x in unique_entity_types]
entity_type_count.sort(key=lambda x: x[1], reverse=True)
post_data.loc[:, "entities_identified"] = pd.Series(all_entity_types, index=indices)
post_data.loc[:, "entities_identified"] = post_data["entities_identified"].apply(
    lambda x: list(set(x))
)
post_data.loc[:, "entities_identified"] = post_data.loc[:, "entities_identified"].apply(
    lambda x: ",".join(x)
)

entity_counts = post_data["entities_identified"].explode().value_counts()
sorted_entities = entity_counts.index.to_list()

# print("Entities identified: ", len(sorted_entities))

# Making sure that the data doesn't have null values in the empty columns
post_data.loc[:, list(text_columns)] = post_data.loc[:, list(text_columns)].fillna(" ")
# [post_data[x].isnull().sum() for x in text_columns]


post_data.loc[:, list(text_columns)]

# removing stopwords and links in the texts
for text_col in text_columns:
    post_data[text_col] = post_data[text_col].apply(
        lambda x: " ".join([word.strip() for word in x.split() if word not in (stop)])
    )


# If lemmatized data is not available, then start lemmatization.
if os.path.isfile(current_dir + "temp_lemmatized_text.csv"):
    post_data.loc[:, list(text_columns)] = pd.read_csv(
        current_dir + "temp_lemmatized_text.csv"
    )
else:
    for text_col in text_columns:
        post_data[text_col] = lemmatize_text(text_col)

    post_data.loc[:, list(text_columns)].to_csv("temp_lemmatized_text.csv", index=False)


# Creating a column for the number of entities identified in each post
post_data["post_sponsored"] = (post_data.sponsor_id.notnull()).astype(int)

# Clipping the dataset.
max_likes = round(np.quantile(post_data.likes, 0.99))
max_comments = round(np.quantile(post_data.comments, 0.99))
max_shares = round(np.quantile(post_data.shares, 0.99))

post_data = post_data[post_data.likes < max_likes].reset_index(drop=True)
post_data = post_data[post_data.comments < max_comments].reset_index(drop=True)
post_data = post_data[post_data.shares < max_shares].reset_index(drop=True)

# Calculating Age of the post and the page
post_data.page_created = pd.to_datetime(post_data.page_created, dayfirst=True)
post_data.post_created = pd.to_datetime(
    post_data["post_created"].str.replace("IST", "")
)
post_data["current_date"] = pd.to_datetime("2022-07-22 09:31:10")
post_data["post_age"] = (post_data.current_date - post_data.post_created).dt.days
post_data["page_age"] = (post_data.current_date - post_data.page_created).dt.days

# grouping public engagements into two broad categories
post_data["positive_reactions"] = (
    post_data["love"] + post_data["haha"] + post_data["wow"] + post_data["care"]
)
post_data["negative_reactions"] = post_data["sad"] + post_data["angry"]
post_data["emoji_reactions"] = (
    post_data["positive_reactions"]
    + post_data["negative_reactions"]
    + post_data["likes"]
)

# Calculating overall views
zero_view_indices = post_data[post_data.post_views == 0].index.to_list()
non_zero_view_indices = post_data[post_data.post_views != 0].index.to_list()

post_data.loc[zero_view_indices, "post_views"] = (
    post_data.loc[
        zero_view_indices,
        [
            "emoji_reactions",
            "comments",
            "shares",
            "total_views",
            "total_views_for_all_crossposts",
        ],
    ]
    .max(axis=1)
    .astype(int)
)
post_data.loc[non_zero_view_indices, "post_views"] = (
    post_data.loc[
        non_zero_view_indices,
        [
            "post_views",
            "emoji_reactions",
            "comments",
            "shares",
            "total_views",
            "total_views_for_all_crossposts",
        ],
    ]
    .max(axis=1)
    .astype(int)
)

# filling followers_at_posting null values
likes_and_following = post_data[["likes_at_posting", "followers_at_posting"]]
null_indices = likes_and_following[
    likes_and_following.followers_at_posting.isnull()
].index.to_list()


# loading the model
with open(current_dir + "followers_model.pkl", "rb") as f:
    lr = joblib.load(f)

needed = likes_and_following.loc[null_indices, "likes_at_posting"].values.reshape(-1, 1)
predicted = lr.predict(needed)
likes_and_following.loc[null_indices, "followers_at_posting"] = predicted
post_data["followers_at_posting"] = likes_and_following["followers_at_posting"].astype(
    int
)

# Cleaning the `type` column
video_types = [
    "Native Video",
    "Video",
    "Live Video Complete",
    "Status",
    "Live Video Scheduled",
    "YouTube",
]
post_data.loc[post_data["type"].isin(video_types), "type"] = "Video"
post_data.loc[:, "type"] = post_data["type"].str.lower()

# Combining the text columns into a single column
post_data.loc[:, list(text_columns)] = post_data.loc[:, list(text_columns)].fillna("")
post_data["post_text"] = (
    post_data["message"]
    + " "
    + post_data["image_text"]
    + " "
    + post_data["link_text"]
    + " "
    + post_data["description"]
)
post_data["post_text"] = post_data.post_text.str.split().apply(
    lambda x: [word.strip() for word in x]
)


# Cleaning the rare_words
all_text = []
for i in tqdm(range(len(post_data))):
    all_text.extend(post_data.post_text[i])

vocab = set(all_text)
word_freq = nltk.FreqDist(all_text)
rare_words = [word for word in vocab if word_freq[word] < 4]
too_frequent_words = [word for word in vocab if word_freq[word] > 155]
words_to_remove = rare_words + too_frequent_words

post_data["post_text"] = post_data["post_text"].apply(
    lambda x: [word for word in x if word not in words_to_remove]
)


post_data.reset_index(drop=True, inplace=True)


100%|██████████| 8786/8786 [00:00<00:00, 123162.33it/s]


In [5]:
dropping_cols = ['sponsor_id', 'sponsor_name', 'sponsor_category', 'page_created', 'post_created', 'current_date',
                 'total_interactions', 'post_created_date', 'post_created_time', 'love', 'haha', 'wow', 'sad', 'angry',
                 'care',  'total_views', 'total_views_for_all_crossposts', 'url', 'link', 'final_link', 
                 'video_share_status', 'is_video_owner?', 'video_length', 'message', 'image_text', 'link_text', 'description',
                 ]

post_data.drop(dropping_cols, axis=1, inplace=True)

In [69]:
post_data.to_csv(current_dir + "cleaned_data.csv", index=False)

## inprogress

In [6]:
post_data.columns

Index(['original_index', 'page_name', 'likes_at_posting',
       'followers_at_posting', 'type', 'likes', 'comments', 'shares',
       'post_views', 'readability', 'readability_lix', 'entropy_scores',
       'perplexity_scores', 'fear', 'anger', 'anticip', 'trust', 'surprise',
       'positive', 'negative', 'sadness', 'disgust', 'joy',
       'entities_identified', 'post_sponsored', 'post_age', 'page_age',
       'positive_reactions', 'negative_reactions', 'emoji_reactions',
       'post_text'],
      dtype='object')

In [7]:
texts = list(post_data.post_text)
id2word = gensim.corpora.Dictionary(post_data.post_text)
corpus = [id2word.doc2bow(doc) for doc in post_data.post_text]

In [36]:
def topic_diversity(ldamodel, topn=25):
    # Get the topn words for each topic
    topic_words = [ldamodel.show_topic(topicid, topn) for topicid in range(ldamodel.num_topics)]
    # Count the unique words across all topics
    unique_words = len({word for topic in topic_words for word, _ in topic})
    # Calculate the topic diversity
    diversity = unique_words / (ldamodel.num_topics * topn)
    return diversity


def train_lda_models(corpus, id2word, texts):
    scores=[]
    for k in tqdm(range(4,15)): # Train LDA on different values of k
        
        ldamodel =gensim.models.LdaMulticore(corpus=corpus,
                                        id2word=id2word,
                                        num_topics=k, 
                                        random_state=42,
                                        chunksize=100,
                                        passes=100,
                                        per_word_topics=True)
        #Calculating the coherence
        if not os.path.isdir(current_dir + f'lda_models/{k}_topics'):
            os.mkdir(current_dir + f'lda_models/{k}_topics')

        ldamodel.save(current_dir + f'lda_models/{k}_topics/lda_model_{k}_topics')


        cm = gensim.models.coherencemodel.CoherenceModel(model=ldamodel, texts=texts, dictionary=id2word, coherence='c_v')
        scores.append((k, cm.get_coherence(), topic_diversity(ldamodel), ldamodel.log_perplexity(corpus)))


    lda_scores = pd.DataFrame(scores, columns=['num_topics', 'coherence', 'topic_diversity', 'perplexity'])
    return lda_scores

def get_topic_distribution(lda_model, corpus):
    topic_distribution = []
    for i in tqdm(range(len(corpus))):
        topic_distribution.append(lda_model.get_document_topics(corpus[i]))
    return topic_distribution


def get_topic_scores(topic_distribution):
    all_topic_scores = []  # Renamed to avoid confusion

    for i in range(len(topic_distribution)):
        dist_scores = []
        dist = topic_distribution[i]
        topics_in_dist = [x[0] for x in dist]
        topic_scores = [
            x[1] for x in dist
        ]  # This is okay, doesn't overwrite all_topic_scores
        for topic_idx in range(12):
            if topic_idx not in topics_in_dist:
                dist_scores.append(np.random.uniform(0.0001, 0.00001))
            else:
                dist_scores.append(topic_scores[topics_in_dist.index(topic_idx)])

        all_topic_scores.append(dist_scores)  # Append to the correct list

    all_topic_scores = pd.DataFrame(
        all_topic_scores, columns=[f"topic_{x}" for x in range(12)]
    )

    return all_topic_scores

# lda_scores = train_lda_models(corpus, id2word, texts)
# lda_scores.sort_values(by=['coherence', 'topic_diversity', 'perplexity'], ascending=False, inplace=True)
# lda_scores

In [24]:
best_k = 12
lda_model = gensim.models.LdaModel.load(current_dir + f'lda_models/{best_k}_topics/lda_model_{best_k}_topics')

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.026154  0.092460       1        1  10.802068
3      0.156937 -0.039784       2        1   9.730212
9     -0.206988  0.015890       3        1   9.057582
8     -0.107353  0.055606       4        1   8.966163
4     -0.154629 -0.205561       5        1   8.611524
6      0.054097 -0.117960       6        1   8.589461
11    -0.171742  0.078462       7        1   8.144974
1      0.111158 -0.246799       8        1   7.562940
2     -0.016825  0.028451       9        1   7.423929
7      0.196725 -0.000095      10        1   7.198908
0      0.015654  0.124969      11        1   7.061999
10     0.149121  0.214362      12        1   6.850239, topic_info=                   Term        Freq       Total Category  logprob  loglift
5875         democratic  418.000000  418.000000  Default  30.0000  30.0000
2517           musician  210.000000  210.000000  Default  29.0000  29.0000
1178            process  233.000000  233.000000  Default  28.0000  28.0000
1369           stranger  201.000000  201.000000  Default  27.0000  27.0000
4384             invest  193.000000  193.000000  Default  26.0000  26.0000
...                 ...         ...         ...      ...      ...      ...
1100                kit   86.477406  121.440294  Topic12  -4.7147   2.3413
4100  kickstarterfunded   74.945139   99.996975  Topic12  -4.8578   2.3925
2189          condition   64.126352   73.218725  Topic12  -5.0137   2.5483
271             brother   66.970950  110.645774  Topic12  -4.9703   2.1788
674             control   64.947919  170.391743  Topic12  -5.0010   1.7164

[531 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2447      6  0.223861     1000
2447      9  0.763762     1000
643       6  0.386764       12
643       7  0.601633       12
948       1  0.873402       15
...     ...       ...      ...
1705      2  0.892158   worlds
1705      9  0.104960   worlds
3132      2  0.990175  writing
1197      1  0.444411     york
1197      4  0.547463     york

[693 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 10, 9, 5, 7, 12, 2, 3, 8, 1, 11])

In [23]:
topic_idx2label = {
    '0':	'Awards and recognitions',
    '1':	'Portable Tech',
    '2':	'Movies and Entertainment',
    '3':	'LifeTech Innovations',
    '4':	'Cultural connections',
    '5':	'Humanitarian',
    '6':	'Media and Events',
    '7':	'Digital and Social trends',
    '8':	'Household Tech',
    '9':	'Small Business',
    '10':	'Tech and Travel Gear'
}

In [30]:
topic_cols = [f"topic_{x}" for x in range(11)]
post_data.drop(topic_cols, axis=1, inplace=True)

In [38]:
topic_distribution = get_topic_distribution(lda_model, corpus)
all_topic_scores = get_topic_scores(topic_distribution)

post_data = pd.concat([post_data, all_topic_scores], axis=1)
# post_data.drop(["post_text", "emoji_reactions"], axis=1, inplace=True)
post_data.to_csv(current_dir + "processed_data.csv", index=False)

100%|██████████| 8786/8786 [00:01<00:00, 7910.36it/s]


In [41]:
top_words_per_topic = []
for topic_id in range(lda_model.num_topics):
    top_words = lda_model.show_topic(topic_id, topn=50)
    top_words_per_topic.append([word for word, _ in top_words])

top_words_per_topic = pd.DataFrame(top_words_per_topic).T
top_words_per_topic.columns = [f"topic_{x}" for x in range(12)]
top_words_per_topic.T.to_csv(current_dir + "top_words_per_topic.csv", index=False)


## Preprocessing